In [1]:
import numpy as np
import pandas as pd
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
# 경고 메시지 무시
warnings.filterwarnings('ignore')


2023-11-27 11:38:04.588404: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-27 11:38:04.637086: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
kobert_model = TFBertModel.from_pretrained('klue/bert-base')

2023-11-24 14:55:53.771039: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-24 14:55:53.781097: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-24 14:55:53.783797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 데이터 로드
data = pd.read_excel("./감성대화말뭉치(최종데이터)_Training.xlsx")


In [11]:
data

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51625,51626,노년,남성,재정,만성질환 무,분노,성가신,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.,아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.,지금 할 수 있는 가장 합리적인 행동은 무엇인가요?,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.,좋은 결과 얻으시길 바랄게요.
51626,51627,노년,여성,재정,만성질환 무,불안,초조한,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.,마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.,어떻게 하면 지금의 기분을 나아지게 할 수 있을까요?,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.,남편과 함께하는 좋은 외출 시간 되시길 바랄게요.
51627,51628,노년,여성,재정,만성질환 무,상처,희생된,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.,주변 사람들은 다 노후 준비도 잘해두었던데 난 어떻게 해야 할지 모르겠어. 막막하기...,지금의 상황에서 할 수 있는 가장 좋은 행동이 무엇일까요?,남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.,좋은 정보 많이 얻으셔서 걱정을 좀 덜으셨으면 좋겠어요.
51628,51629,노년,여성,대인관계,만성질환 무,불안,걱정스러운,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.


In [3]:
# 가사 단어 100토큰으로줄이기
for i in range (len(data["사람문장1"])):
    list1=list(data["사람문장1"][i].split(" "))
    list1= list1[:100]
    data["사람문장1"][i]=" ".join(list1)

In [4]:
data=data[["감정_대분류","사람문장1"]]

In [5]:
data

,감정_대분류,사람문장1
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...,...
51625,분노,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.
51626,불안,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.
51627,상처,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.
51628,불안,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.


In [6]:
data['감정_대분류'].unique()

array(['분노', '기쁨', '불안', '당황', '슬픔', '상처'], dtype=object)

In [6]:
# 감정 태그를 숫자로 매핑
tag_mapping = {tag: idx for idx, tag in enumerate(data['감정_대분류'].unique())}
inverse_tag_mapping = {v: k for k, v in tag_mapping.items()}
data['감정_대분류_encoded'] = data['감정_대분류'].map(tag_mapping)

# 데이터 분할
train,test= train_test_split(data[["사람문장1",'감정_대분류_encoded']], test_size=0.2, random_state=42)

In [12]:
train

,사람문장1,감정_대분류_encoded
21798,오늘 학교 화장실에서 친구들이 나를 가두고 변기 물을 뿌려서 구역질났어.,0
38273,요즘 자꾸 깜박깜박 뭘 자주 잊어버려.,4
48537,갱년기에 접어들면서 나 자신이 하찮게 느껴지고 불안감에 다른 사람들을 믿지 못하게 됐어.,5
7815,회사 어떤 여직원이 자기보다 나이가 많은 줄 알았다며 나에게 이야기를 하는 거 있지...,5
42449,시험공부 하느라 친구들의 연락을 다 거절해서 미안하고 친구들이 날 싫어할까 봐 두려워.,3
...,...,...
11284,나는 친구가 너무 없어.,2
44732,불편한 다리 때문에 더는 일을 못 하니까 아무도 날 찾지 않아. 삶이 허탈하게 느껴져.,5
38158,전에는 일본에서 황혼이혼이라는 말이 유행하더니 이제는 우리나라도 마찬가지인가봐. 그...,2
860,나랑 대화하는 걸 좋아하는 할머니가 있는데 그분 덕분에 나도 기분이 좋아.,1


In [7]:
x_train = tokenizer(
    text=train.사람문장1.tolist(),
    add_special_tokens=True,
    max_length=50,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


x_test = tokenizer(
    text=test.사람문장1.tolist(),
    add_special_tokens=True,
    max_length=50,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

NameError: name 'tokenizer' is not defined

In [14]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [15]:
max_len = 50
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = kobert_model(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(256, activation='relu')(out)
out = tf.keras.layers.Dropout(0.4)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(6,activation = 'softmax')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True
# for training bert our lr must be so small

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 50)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  110617344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 50,                                            

In [17]:
optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    
    clipnorm=1.0)

# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [18]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(train.감정_대분류_encoded),
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(test.감정_대분류_encoded)),
    epochs=50,
    batch_size=64)

Epoch 1/50
646/646 [==============================] - 56s 65ms/step - loss: 1.3196 - balanced_accuracy: 0.4906 - val_loss: 1.1842 - val_balanced_accuracy: 0.5644
Epoch 2/50
646/646 [==============================] - 41s 63ms/step - loss: 1.1146 - balanced_accuracy: 0.5859 - val_loss: 1.1530 - val_balanced_accuracy: 0.5714
Epoch 3/50
646/646 [==============================] - 40s 63ms/step - loss: 1.0117 - balanced_accuracy: 0.6273 - val_loss: 1.1407 - val_balanced_accuracy: 0.5788
Epoch 4/50
  7/646 [..............................] - ETA: 36s - loss: 0.8836 - balanced_accuracy: 0.7009

KeyboardInterrupt: 

In [19]:
# 예측 결과를 원래 태그로 변환
predictions = model.predict(
   {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_tags = np.argmax(predictions, axis=1)
# predicted_tags = [inverse_tag_mapping[tag] for tag in predicted_tags]

# print("Predicted Tags:", predicted_tags)


323/323 [==============================] - 8s 17ms/step


In [20]:
test["predict"]=predicted_tags

In [21]:
test

,사람문장1,감정_대분류_encoded,predict
44927,요즘 주변에서 부모님들과 여행 다녀온 아이들을 보면 너무 부러워.,3,3
25924,나와 남편은 귀가 얇아서 사기를 잘 당해서 재정이 취약하고 늘 불안해.,2,2
47384,오늘도 폐지 주워 오천 원밖에 못 벌었어. 앞으로 어떻게 살지 왜 난 이따위 사람일까.,0,0
8570,우리 언니는 왜 나한테 사사건건 간섭일까? 스트레스다 정말.,2,2
3227,오늘 이직 면접을 보고 왔는데 괜히 보고 왔나 봐. 면접 간 게 후회돼.,2,4
...,...,...,...
19002,옆집 할아버지가 입원을 했는데 마침 우리 집 농기계가 고장이 났어. 말도 없이 빌려...,3,3
13152,내 친구들을 볼 때마다 자존감이 낮아져.,2,0
28051,잘못된 종류의 빵을 사 가서 폭력을 당하는 모습을 좋아하던 아이한테 들켰어.,3,3
47896,노후 준비를 아직 하지 못한 내가 싫어.,0,0


In [16]:
from sklearn.metrics import classification_report

# 예측 결과와 실제 레이블을 가지고 classification report 생성
y_true = test.감정_대분류_encoded
y_pred =np.argmax(predictions, axis=1)

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.51      0.55      0.53      1831
           1       0.91      0.83      0.87      1206
           2       0.57      0.54      0.55      1891
           3       0.57      0.43      0.49      1718
           4       0.52      0.55      0.54      1843
           5       0.43      0.53      0.48      1837

    accuracy                           0.56     10326
   macro avg       0.59      0.57      0.58     10326
weighted avg       0.57      0.56      0.56     10326



In [8]:
data["감정_대분류_encoded"].value_counts()

감정_대분류_encoded
2    9320
0    9160
5    9143
4    9125
3    8756
1    6126
Name: count, dtype: int64

In [9]:
data["감정_대분류"].value_counts()

감정_대분류
불안    9320
분노    9160
상처    9143
슬픔    9125
당황    8756
기쁨    6126
Name: count, dtype: int64

In [18]:
val_data=pd.read_excel("./감성대화말뭉치(최종데이터)_Validation.xlsx")

In [20]:
# 감정 태그를 숫자로 매핑
tag_mapping = {tag: idx for idx, tag in enumerate(val_data['감정_대분류'].unique())}
inverse_tag_mapping = {v: k for k, v in tag_mapping.items()}
val_data['감정_대분류_encoded'] = val_data['감정_대분류'].map(tag_mapping)


In [19]:
val_train = tokenizer(
    text=val_data.사람문장1.tolist(),
    add_special_tokens=True,
    max_length=50,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [21]:
# 예측 결과를 원래 태그로 변환
predictions = model.predict(
   {'input_ids':val_train['input_ids'],'attention_mask':val_train['attention_mask']})
predicted_tags = np.argmax(predictions, axis=1)
predicted_tags = [inverse_tag_mapping[tag] for tag in predicted_tags]

print("Predicted Tags:", predicted_tags)


208/208 [==============================] - 4s 18ms/step
Predicted Tags: ['당황', '기쁨', '당황', '당황', '당황', '당황', '당황', '당황', '당황', '당황', '당황', '당황', '분노', '분노', '분노', '기쁨', '분노', '분노', '분노', '분노', '분노', '분노', '분노', '분노', '상처', '불안', '기쁨', '상처', '기쁨', '기쁨', '기쁨', '기쁨', '상처', '기쁨', '기쁨', '분노', '슬픔', '당황', '슬픔', '슬픔', '슬픔', '슬픔', '기쁨', '슬픔', '슬픔', '슬픔', '슬픔', '슬픔', '기쁨', '기쁨', '기쁨', '기쁨', '상처', '불안', '기쁨', '기쁨', '당황', '상처', '기쁨', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '분노', '불안', '불안', '불안', '분노', '분노', '분노', '분노', '분노', '분노', '분노', '분노', '분노', '분노', '분노', '분노', '불안', '불안', '불안', '당황', '불안', '상처', '불안', '불안', '불안', '불안', '당황', '불안', '상처', '상처', '상처', '상처', '상처', '상처', '상처', '상처', '상처', '상처', '당황', '상처', '분노', '분노', '분노', '분노', '분노', '분노', '분노', '상처', '상처', '분노', '분노', '슬픔', '슬픔', '슬픔', '슬픔', '슬픔', '슬픔', '슬픔', '슬픔', '슬픔', '슬픔', '슬픔', '슬픔', '상처', '불안', '불안', '불안', '불안', '분노', '불안', '불안', '불안', '불안', '불안', '상처', '불안', '기쁨', '기쁨', '기쁨', '기쁨', '기쁨', '기쁨', '기쁨', '기쁨', '기쁨', '불안', '불안

In [8]:
test["예측"]=predicted_tags

NameError: name 'predictions' is not defined

In [22]:
from sklearn.metrics import classification_report

# 예측 결과와 실제 레이블을 가지고 classification report 생성
y_true = val_data.감정_대분류_encoded
y_pred =np.argmax(predictions, axis=1)

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.06      0.08      0.07      1113
           1       0.01      0.01      0.01      1003
           2       0.09      0.09      0.09      1048
           3       0.02      0.02      0.02      1213
           4       0.08      0.07      0.07      1257
           5       0.58      0.67      0.62      1007

    accuracy                           0.15      6641
   macro avg       0.14      0.16      0.15      6641
weighted avg       0.13      0.15      0.14      6641



In [23]:
model.save_weights('./model/emotion/emotionkr_weights')
model.save('./model/emotion/emotionkr.h5')

In [24]:
save_directory = "./bert_model_directory/klue_bert_base"

# 토크나이저 저장
tokenizer.save_pretrained(save_directory)

# 모델 저장
kobert_model.save_pretrained(save_directory)